# scraping wikipedia page using Pandas

####    1. install prerequesties lxml, html5lib and beautifulsoup4

In [1]:
pip install BeautifulSoup4

     |████████████████████████████████| 112kB 3.1MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
# install lxml so we can read web pages
pip install lxml

     |████████████████████████████████| 5.8MB 5.3MB/s eta 0:00:01     |███████████████████████████▍    | 4.9MB 5.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import requests 

In [4]:
### put the url in variable
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


wiki_page= requests.get(url).text

soup= BeautifulSoup(wiki_page, "html5lib")

#print(soup.prettify())


#### extracting table from th webpage

In [5]:

postcode=[]
borough=[]
neighborhood=[]

rtable= soup.find('table', {'class':'wikitable sortable'})

for row in rtable.find_all('tr'):
    cells=row.find_all('td')
    if len(cells)==3:
        postcode.append(cells[0].find(text = True))
        
        borough.append(cells[1].find(text=True))
        neighborhood.append(cells[2].find(text=True))




#### transforming our table int pandas data frame

In [6]:
df=pd.DataFrame(postcode, columns=['Postalcode'])
df['Borough']= borough
df['Neighborhood']= neighborhood

df.head(5)
#df.shape

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [7]:
# remove \n from our data frame

df=df.replace('\n', '', regex= True)

#### remove not assigned rows

In [8]:
# drop N\A values
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]


In [9]:
#combine neighborhoods that have same postal code area. 

df2= df.groupby('Postalcode').agg(lambda x: ','.join(x))


In [13]:

df2.shape

(103, 3)

In [14]:
# reset index and show the dataframe
df2=df2.reset_index()
df2.head()


,index,Postalcode,Borough,Neighborhood
0,0,M1B,Scarborough,"Malvern, Rouge"
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,3,M1G,Scarborough,Woburn
4,4,M1H,Scarborough,Cedarbrae
